In [1]:
!pip install ftfy regex tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-kge7xqix
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-kge7xqix


In [ ]:
from google.colab import files
files.upload()

In [27]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print('done')

done


In [23]:
def get_logits(image_features, text_features):
  # normalized features
  image_features = image_features / image_features.norm(dim=1, keepdim=True)
  text_features = text_features / text_features.norm(dim=1, keepdim=True)

  # cosine similarity as logits
  logit_scale = model.logit_scale.exp()
  logits_per_image = logit_scale * image_features @ text_features.t()
  logits_per_text = logits_per_image.t()

  return logits_per_image, logits_per_text

In [14]:
image = preprocess(Image.open("melon.png")).unsqueeze(0).to(device)
text = clip.tokenize(["red", "a city", "a dog"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs) 

Label probs: [[0.611  0.1779 0.2112]]


In [25]:
logits_per_image, logits_per_text = get_logits(image_features, text_features)
print(logits_per_image.shape)
print(logits_per_text.shape)

torch.Size([1, 3])
torch.Size([3, 1])


In [26]:
logits_per_image.softmax(dim=-1)

tensor([[0.6108, 0.1779, 0.2112]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SoftmaxBackward0>)